In [1]:
!pip install langchain[openai]
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-groq
!pip install beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 4.5 MB/s eta 0:00:00


In [4]:
!pip install python-dotenv
!pip install cassio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 32.9 MB/s eta 0:00:00


In [5]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings,OllamaEmbeddings
from langchain.vectorstores.cassandra import Cassandra
import cassio
from dotenv import load_dotenv
load_dotenv()

False

In [13]:

# Connection of ASTRA DB(Datastax)
groq_api_key = os.getenv("GROQ_API_KEY")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [14]:
import bs4
loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)

In [16]:
# Convert Data into vectors and store in ASTRA DB
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

embedding = OpenAIEmbeddings()
astra_vector_store = Cassandra(embedding = embedding,
                               table_name = "qa_mini_demo",
                               session= None,
                               keyspace = None)


In [18]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(documents)

print("Inserted %i headlines." %len(documents))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 66 headlines.


In [19]:
llm = ChatGroq(groq_api_key = groq_api_key,model_name = "mixtral-8x7b-32768")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>

Question: {input}""")

In [20]:
astra_vector_index.query("Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique",llm=llm)

"Based on the provided context, Tree of Thoughts (Yao et al., 2023) is a method that builds upon Chain of Thought (CoT) by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts. The task decomposition in Tree of Thoughts can be done using a large language model (LLM) through simple prompting, task-specific instructions, or human inputs.\n\nChain of Hindsight (CoH; Liu et al., 2023) is a separate method that focuses on improving the model's own outputs by presenting it with a sequence of past outputs, each annotated with feedback. This process is done through supervised fine-tuning, where the model is trained to condition on a sequence prefix and produce better outputs based on the feedback sequence.\n\nReAct (Yao et al., 2023) is yet another method that integrates reasoning and acting within a large language model (LLM). It extends the action space to include both task-specific discrete actions (enabling the LLM to interact with the environment)

In [22]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

retriever = astra_vector_store.as_retriever()

document_chain = create_stuff_documents_chain(llm,prompt)
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [23]:
response=retrieval_chain.invoke({"input":"Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique"})
response

{'input': 'Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique',
 'context': [Document(id='ec9ea30464fc4ef297602b6b9cefe875', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'),
  Document(id='4ad35b04cc5849fab2082891188fa834', metadata={'source': 'https://lilianweng.github.io/pos

In [24]:
response["answer"]

'for enhancing model performance on complex tasks. The model is instructed to "think step by step" to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. This process transforms big tasks into multiple manageable tasks and sheds light into an interpretation of the model\'s thinking process.\n\nIn the Tree of Thoughts (Yao et al. 2023) approach, this CoT technique is extended by exploring multiple reasoning possibilities at each step. The problem is first decomposed into multiple thought steps, and multiple thoughts are generated per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search), with each state evaluated by a classifier (via a prompt) or majority vote.\n\nTask decomposition in the Tree of Thoughts can be done in three ways:\n1. By LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?"\n2. By using task-specific instructions; e.g. "Write a 